# DNN with tensorflow
## 1. Training multi layer perceptron with high-level API 

We can get rough intuition of DNN from coding with high-level API. TF.Learn is compatible with scikitlearn.

With DNNClassifier, we are going to use, Training DNN consisting of hidden layer and softmax output layer.

Next code trains 2 hidden layers and 10 output softmax layers.

In [7]:
import tensorflow as tf
import numpy as np
'''
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes=10, feature_columns=feature_cols)

dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)
dnn_clf.fit(X_train, y_Train, batch_size=50, steps=10000)
'''

'\nfeature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)\ndnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes=10, feature_columns=feature_cols)\n\ndnn_clf = tf.contrib.learn.SKCompat(dnn_clf)\ndnn_clf.fit(X_train, y_Train, batch_size=50, steps=10000)\n'

# 2. Training deep neural network with low-level API

If we want to control more detail option in network, which is always in practical problem, using low-level API is better.

I am going to build neural network for MNIST dataset with mini-batch gradient descent.

There are two step. First, constructing graph. And running.

##### Input, output and number of neurons

In [27]:
tf.reset_default_graph()

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

##### Placeholder 

In [28]:
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name="X")
Y = tf.placeholder(tf.int64, shape = (None), name="Y")

##### # One layer

In [29]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs+n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

##### Multi layers with One layer

In [30]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs") # Before softmax

Actually!

we don't have to define functino 'neuron_layer'. 

By using tf.layers.dense() function, we can easily make parameters, W and b, and even provide regularization and normalization method.

Use dense()! instead of neuron_layer().

In [31]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

##### Compute cost

Model is ready. Then, we need to define cost. I am going to use simple cross entropy.

In [32]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

##### Optimizer

In [33]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

##### Evaluation

In [35]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, Y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

##### Init and Saver

In [37]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# Time to run

In [40]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [41]:
n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, Y: Y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, Y: Y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           Y: mnist.validation.labels})
        print(epoch, "Train accuracy:", acc_train, "Validation accracy:", acc_val)
    
    save_path = saver.save(sess, "./my_model_final.ckpt")

##### Apply to predict

In [45]:
'''
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    X_new_scaled = [...]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    Y_pred = np.argmax(Z, axis=1)
'''

'\nwith tf.Session() as sess:\n    saver.restore(sess, "./my_model_final.ckpt")\n    X_new_scaled = [...]\n    Z = logits.eval(feed_dict={X: X_new_scaled})\n    Y_pred = np.argmax(Z, axis=1)\n'